In [27]:
from platform import python_version
import os
import shutil
import numpy as np
import pandas as pd
import PIL.Image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from matplotlib.font_manager import findfont, FontProperties



from matplotlib.font_manager import FontProperties

import keras

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D  
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

print( 'Python Version: ', python_version() )
print( 'TensorFlow Version: ', tf.__version__ )
print( 'Keras Version: ', tf.keras.__version__ ) 

Python Version:  3.9.13
TensorFlow Version:  2.9.0
Keras Version:  2.9.0


In [31]:
path = 'D:/download/최종dataset'

In [32]:

a=0
b=0

for item in os.listdir(inputFolder):
    a += 1
    for i in os.listdir(inputFolder + '/' + item):
        b +=1


print('총: ' + str(a) + ' 단어(폴더)/합계: ' + str(b) )
print('평균 개수: ' + str(b/a))


총: 1817 단어(폴더)/합계: 181700
평균 개수: 100.0


In [33]:
Num_Classes = 1817
#len(os.listdir())
Image_Size = ( 120, 120 )
Epochs = 150
Batch_Size = 16
SplitRatio = 0.2

In [36]:
Train_Data_Genetor = ImageDataGenerator(
    rescale=1./255,                 # 이미지 픽셀 0~1로 조정
    validation_split=SplitRatio,    # 훈련데이터 0.8 / 검증데이터 0.2
    rotation_range=10,              # 이미지를 10도 범위 내에서 회전
    width_shift_range=0.1,          # (가로의 0.1 범위 내에서) 가로 방향으로 무작위 이동
    height_shift_range=0.1,         # (세로의 0.1 범위 내에서) 세로 방향으로 무작위 이동
    zoom_range=0.1,                 # 0.1배 무작위 확대/축소
    horizontal_flip=False           # 수평뒤집기 x
)

In [37]:
Train_Generator = Train_Data_Genetor.flow_from_directory( path ,
                                                          target_size = Image_Size,
                                                          batch_size = Batch_Size,
                                                          class_mode = 'categorical',
                                                          shuffle = True, 
                                                          subset = 'training' )

Found 145360 images belonging to 1817 classes.


In [38]:
Val_Generator = Train_Data_Genetor.flow_from_directory( path ,
                                                        target_size = Image_Size,
                                                        batch_size = Batch_Size,
                                                        class_mode = 'categorical',
                                                        shuffle = True, 
                                                        subset = 'validation' )

Found 36340 images belonging to 1817 classes.


In [43]:
CNN = Sequential()
CNN.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(150,150,3), kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())

CNN.add(MaxPooling2D(3,3))

CNN.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())

CNN.add(MaxPooling2D(3,3))

CNN.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())

CNN.add(MaxPooling2D(3,3))

CNN.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())
CNN.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
CNN.add(BatchNormalization())

CNN.add(MaxPooling2D(3,3))

CNN.add(GlobalAveragePooling2D())
CNN.add( Dense( Num_Classes, activation = 'softmax', name = 'Softmax' ) )

In [44]:
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 batch_normalization_8 (Batc  (None, 150, 150, 32)     128       
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 batch_normalization_9 (Batc  (None, 150, 150, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 50, 50, 32)       0         
 2D)                                                             
                                                      

In [46]:
callbacks = [
    # monitor='val_accuracy'  ==>  유효성검사 정확도를 모니터링해서
    # patience=5  ==> 5번 이상 개선되지 않으면 학습 중지
    # mode='max' ==> 유효성 검사 정확도를 최대화
    # verbose=1 ==> 교육 진행 및 조기 중지 조건에 대한 자세한 출력 메시지를 생성
    EarlyStopping(monitor='val_accuracy', patience=5, mode='max', verbose=1),
    ModelCheckpoint('D:/model/dh_CNN_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
]

In [49]:
opt = keras.optimizers.Adam(learning_rate = 0.001)
CNN.compile( optimizer = opt,
             loss = 'categorical_crossentropy', 
             metrics = ['accuracy'] )

In [ ]:
History = CNN.fit( Train_Generator,
                   steps_per_epoch = Train_Generator.samples//Batch_Size,
                   validation_data = Val_Generator,
                   validation_steps = Val_Generator.samples//Batch_Size,
                   epochs = Epochs,
                 callbacks=callbacks )

Epoch 1/150
9085/9085 [==============================] - ETA: 0s - loss: 3.1310 - accuracy: 0.4113
Epoch 1: val_accuracy improved from -inf to 0.67856, saving model to dh_CNN_model.h5
9085/9085 [==============================] - 3768s 415ms/step - loss: 3.1310 - accuracy: 0.4113 - val_loss: 1.2813 - val_accuracy: 0.6786
Epoch 2/150
9085/9085 [==============================] - ETA: 0s - loss: 0.6350 - accuracy: 0.8308
Epoch 2: val_accuracy improved from 0.67856 to 0.78729, saving model to dh_CNN_model.h5
9085/9085 [==============================] - 1116s 123ms/step - loss: 0.6350 - accuracy: 0.8308 - val_loss: 0.8403 - val_accuracy: 0.7873
Epoch 3/150
9084/9085 [============================>.] - ETA: 0s - loss: 0.3997 - accuracy: 0.8909
Epoch 3: val_accuracy improved from 0.78729 to 0.84024, saving model to dh_CNN_model.h5
9085/9085 [==============================] - 541s 60ms/step - loss: 0.3996 - accuracy: 0.8909 - val_loss: 0.6183 - val_accuracy: 0.8402
Epoch 4/150
9085/9085 [=======

In [ ]:
Train_Accuracy = History.history['accuracy']
Val_Accuracy = History.history['val_accuracy']
Train_Loss = History.history['loss']
Val_Loss = History.history['val_loss']
epochs_range = range(Epochs)

plt.figure( figsize=(14,4) )
plt.subplot( 1,2,1 )
plt.plot( range( len(Train_Accuracy) ), Train_Accuracy, label='Train' ) 
plt.plot( range( len(Val_Accuracy) ), Val_Accuracy, label='Validation' ) 
plt.legend( loc='lower right' )
plt.title( 'Accuracy' )

plt.subplot( 1,2,2 )
plt.plot( range( len(Train_Loss) ), Train_Loss, label='Train' )
plt.plot( range( len(Val_Loss) ), Val_Loss, label='Validation' )
plt.legend( loc='upper right' )
plt.title( 'Loss')

plt.show()

In [34]:
#model_path = 'D:/model'

In [35]:
#os.chdir( model_path )
#CNN.save( 'CNN_Model3.h5' )

In [ ]:
CNN = tf.keras.models.load_model(model_path+'dh_CNN_model.h5')

In [ ]:
History = CNN.fit( Train_Generator,
                   steps_per_epoch = Train_Generator.samples//Batch_Size,
                   validation_data = Val_Generator,
                   validation_steps = Val_Generator.samples//Batch_Size,
                   epochs = Epochs,
                 callbacks=callbacks )